In [1]:
import requests
import pprint
import json
import pandas as pd

url = 'http://apis.data.go.kr/1741000/NSIService/getFcltsCheckResultInfoSearch'
params ={'serviceKey' : '3ouN4EKp4qGz+V76EbDHKehnbp5sYL0o19tpl5fAl2Q7s4ZosClGRfc1ENwk+2Px4QUPi4gCuCHGuG3kXFrs9w==',
        'pageNo' : '1',
        'numOfRows' : '10',
        'resultType' : 'json',
        'check_year' : '2020',
        'fclts_nm' : '어린이집',
        'ldong_addr_mgpl_dg_cd' : '41',
        'ldong_addr_mgpl_sggu_cd' : '41195',
        'ldong_addr_mgpl_sggu_emd_cd' : '41195000',
        'last_modf_dt' : '20200101' }

response = requests.get(url, params=params)
contents = response.text
# pp = pprint.PrettyPrinter(indent =4)
# pp.pprint(response.content)

json_ob = json.loads(contents)
body = json_ob['response']['body']['item']
body = pd.json_normalize(body)

In [2]:
body

,ifCudFlag,safeFcltsMngNo,chckRsltSn,fllwManagtCd,fllwManagtNm,safeClCd,chckTrgTyNm,jrsdRelmNm,fcltsNm,eqpNm,...,chckEndDe,pntmtrCn,fllwManagtCn,chckYear,fllwManagtSttusCd,fllwManagtSttusNm,fctlsAdres,latitude,longitude,lastModfDt
0,I,13200000061815,1,,,0210408,어린이보호구역,교통시설,도당동 근로복지어린이집,,...,20200623,,,2020,,점검완료,경기도 부천시 도당동 46,37.5181605781145,126.787657417824,2020-12-02 11:43:59
1,I,13200000061816,1,,,0210408,어린이보호구역,교통시설,새중동어린이집,,...,20200623,,,2020,,점검완료,경기도 부천시 중동 710-4,37.4949657198562,126.765566874769,2020-12-02 11:43:59
2,I,13200000061817,1,,,0210408,어린이보호구역,교통시설,숙명키즈어린이집,,...,20200623,,,2020,,점검완료,경기도 부천시 역곡동 103-6,37.4904239563354,126.811091827007,2020-12-02 11:43:59
3,I,13200000061830,1,,,0210408,어린이보호구역,교통시설,재원어린이집,,...,20200623,,,2020,,점검완료,경기도 부천시 역곡동 115-1,37.4885188473775,126.814432931513,2020-12-02 11:43:59
4,I,13200000061831,1,,,0210408,어린이보호구역,교통시설,큰나무어린이집,,...,20200623,,,2020,,점검완료,경기도 부천시 약대동 47-16,37.5140631887315,126.768066731182,2020-12-02 11:43:59


In [3]:
# 시도코드
{'서울특별시': 11,'부산광역시': 21,
 '대구광역시': 22,'인천광역시': 23,
 '광주광역시': 24,'대전광역시': 25,
 '울산광역시': 26,'세종특별자치시': 29,
 '경기도': 31,'강원도': 32,'충청북도': 33,
 '충청남도': 34,'전라북도': 35,'전라남도': 36,
 '경상북도': 37,'경상남도': 38,'제주특별자치도': 39}

{'서울특별시': 11,
 '부산광역시': 21,
 '대구광역시': 22,
 '인천광역시': 23,
 '광주광역시': 24,
 '대전광역시': 25,
 '울산광역시': 26,
 '세종특별자치시': 29,
 '경기도': 31,
 '강원도': 32,
 '충청북도': 33,
 '충청남도': 34,
 '전라북도': 35,
 '전라남도': 36,
 '경상북도': 37,
 '경상남도': 38,
 '제주특별자치도': 39}

# 이재민 임시주거시설 정보 데이터 만들기

In [4]:
import requests
import pprint
import json
import pandas as pd
import os

vin_df = pd.DataFrame()
pageNo = 1
while pageNo <= 16:
        pageNo = str(pageNo)
        url = 'http://apis.data.go.kr/1741000/TemporaryHousingFacilityVictim3/getTemporaryHousingFacilityVictim1List'
        params ={'serviceKey' : '3ouN4EKp4qGz+V76EbDHKehnbp5sYL0o19tpl5fAl2Q7s4ZosClGRfc1ENwk+2Px4QUPi4gCuCHGuG3kXFrs9w==',
                'pageNo' : pageNo,
                'numOfRows' : '1000',
                'type' : 'json' }

        response = requests.get(url, params=params)
        contents = response.text
        json_ob = json.loads(contents)
        body = json_ob['TemporaryHousingFacilityVictim'][1]['row']
        body = pd.json_normalize(body)
        vin_df = pd.concat([vin_df,body], axis = 0)
        pageNo = int(pageNo) + 1
        
columns = ['ctprvn_nm', 'sgg_nm', 'acmdfclty_se_nm', 'vt_acmdfclty_nm', 'dtl_adres', 'fclty_ar', 'vt_acmd_psbl_nmpr', 'mngps_nm', 'mngps_telno', 'xcord', 'ycord']
vin_df2 = vin_df[columns]
vin_df2.columns = ['시도명', '시군구명', '시설구분', '시설명', '상세주소','시설면적', '주거능력','관리부서','지자체담당자연락처', '경도', '위도']
vin_df2
# vin_df2.reset_index(drop = True).to_csv('C:/pythonproject/streamlit/using_data/이재민 임시주거시설 정보.csv', index = False, encoding = 'utf-8-sig')
# json_ob = json.loads(contents)
# body = json_ob['response']['body']['item']
# body = pd.json_normalize(body)

,시도명,시군구명,시설구분,시설명,상세주소,시설면적,주거능력,관리부서,지자체담당자연락처,경도,위도
0,충청남도,예산군,마을회관,신례원2리 마을회관,충청남도 예산군 예산읍 벚꽃로464번길 49,302,116,신원철,010-9420-45777,126.851709,36.718554
1,충청남도,예산군,학교,신례원초등학교,충청남도 예산군 예산읍 신례원로 179,6893,2651,현정실,041-334-2164,126.842700,36.726120
2,충청남도,천안시 서북구,경로당,천흥1리 경로당,충청남도 천안시 서북구 성거읍 천흥3길 13(성거국민주택),72,27,이인군,,127.201468,36.872021
3,충청남도,천안시 서북구,경로당,성거복지회관 대한노인회 성거읍분회,충청남도 천안시 서북구 성거읍 천흥1길 16,328,126,이인군,,127.200759,36.869382
4,전라북도,장수군,"공공시설(국·공립도서관, 공립병원, 시·도민회관, 구민회관 주민체육시설, 노인병원,...",장수군 보건의료원,전라북도 장수군 장수읍 장천로 245,412,156,장수군청,,127.519811,35.654604
...,...,...,...,...,...,...,...,...,...,...,...
28,경상남도,창원시 마산회원구,학교,마산중앙초등학교,경상남도 창원시 마산회원구 구암2동 9-2,1403,425,행정실,055-297-6244,128.602283,35.248479
29,경기도,평택시,학교,동삭초등학교,경기도 평택시 동삭동 332-10,394,151,신지원,01020666257,127.097503,37.018651
30,세종특별자치시,세종시,관공서,해밀동 복합커뮤니티센터,세종특별자치시 세종시 해밀동 325,1077,414,,,127.268013,36.527121
31,서울특별시,서대문구,"공공시설(국·공립도서관, 공립병원, 시·도민회관, 구민회관 주민체육시설, 노인병원,...",북아현문화체육센터,서울특별시 서대문구 북아현동 1012-2,1555,471,변종진,,125.000000,35.000000


# 구호물자

In [6]:
df = pd.read_csv("C:/pythonproject/streamlit/using_data/행정안전부_재해구호물자 정보_04_30_2020.csv", encoding = 'cp949')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/pythonproject/streamlit/using_data/행정안전부_재해구호물자 정보_04_30_2020.csv'

In [ ]:
df['시도명'] = df['주소'].str.split(' ').str[0]
df['시군구명'] = df['주소'].str.split(' ').str[1]

In [ ]:
df.to_csv('C:/pythonproject/streamlit/using_data/구호물자정보.csv', index = False, encoding = 'utf-8-sig')

# 주민대피시설 및 대피가능 인구 통계 정보

In [7]:
import requests
import pprint
import json
import pandas as pd
url = 'http://apis.data.go.kr/1741000/ResidentEvacuateYearAfter2017'
params ={'serviceKey' : '3ouN4EKp4qGz+V76EbDHKehnbp5sYL0o19tpl5fAl2Q7s4ZosClGRfc1ENwk+2Px4QUPi4gCuCHGuG3kXFrs9w==',
        'pageNo' : '1',
        'numOfRows' : '10'}
response = requests.get(url, params=params)
content = response.text

In [8]:
pp = pprint.PrettyPrinter(indent = 4)

In [9]:
pp.pprint(content)

('<?xml version="1.0" encoding="UTF-8"?>\n'
 '<soapenv:Envelope '
 'xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">\n'
 '    <soapenv:Body>\n'
 '        <soapenv:Fault>\n'
 '            <faultcode>soapenv:Server</faultcode>\n'
 '            <faultstring>Policy Falsified</faultstring>\n'
 '            '
 '<faultactor>http://apis.data.go.kr/1741000/ResidentEvacuateYearAfter2017?serviceKey=3ouN4EKp4qGz%2BV76EbDHKehnbp5sYL0o19tpl5fAl2Q7s4ZosClGRfc1ENwk%2B2Px4QUPi4gCuCHGuG3kXFrs9w%3D%3D&amp;pageNo=1&amp;numOfRows=10</faultactor>\n'
 '            <detail>\n'
 '                <l7:policyResult\n'
 '                    status="Service Not Found.  The request may have been '
 'sent to an invalid URL, or intended for an unsupported operation." '
 'xmlns:l7="http://www.layer7tech.com/ws/policy/fault"/>\n'
 '            </detail>\n'
 '        </soapenv:Fault>\n'
 '    </soapenv:Body>\n'
 '</soapenv:Envelope>\n')


In [10]:
from PIL import Image
def load_image(image_file):
    img = Image.open(image_file)
    return img
load_image('C:/Users/82106/Documents/PythonProject/streamlit/LDM_11620/LDM_11620.img')

UnidentifiedImageError: cannot identify image file 'C:/Users/82106/Documents/PythonProject/streamlit/LDM_11620/LDM_11620.img'

In [ ]:
# Python3 샘플 코드 #

import pprint
import requests
from os import name
import xml.etree.ElementTree as et
import pandas as pd
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

# # 깔끔한 출력 위한 코드
# pp = pprint.PrettyPrinter(indent=4)
# print(pp.pprint(content))


url = 'http://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire'
params ={'serviceKey' : '3ouN4EKp4qGz+V76EbDHKehnbp5sYL0o19tpl5fAl2Q7s4ZosClGRfc1ENwk+2Px4QUPi4gCuCHGuG3kXFrs9w==', 'STAGE1' : '서울특별시', 'STAGE2' : '중랑구', 'pageNo' : '1', 'numOfRows' : '100' }

response = requests.get(url, params=params)
content = response.text

### xml을 DataFrame으로 변환하기 ###
#bs4 사용하여 item 태그 분리

xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('item')

# 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

# xml 안의 데이터 수집
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

#xml값 DataFrame으로 만들기
corona_df = pd.DataFrame(row_list, columns=name_list)
corona_df = corona_df[['hvidate','hvec','hvoc','hvgc','hvamyn','dutyName','dutyTel3']]
corona_df.columns = ['정보 업데이트 일시','응급실 가용현황', '수술실 가용현황', '입원실 가용현황', '구급차 가용여부', '기관명', '연락처']
corona_df

In [11]:

'''
hvidate : 입력일시
hvec : 응급실 가용현황
hvoc : 수술실 가용현황
hvgc : 입원실 가용현황
hvamyn : 구급차가용여부
dutyname : 기관명
hv10 : VENTI(소아)
dutytel3 : 응급실전화
'''

'\nhvidate : 입력일시\nhvec : 응급실 가용현황\nhvoc : 수술실 가용현황\nhvgc : 입원실 가용현황\nhvamyn : 구급차가용여부\ndutyname : 기관명\nhv10 : VENTI(소아)\ndutytel3 : 응급실전화\n'

In [12]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import json
import pandas as pd
from datetime import datetime,timedelta
import warnings
import xmltodict # 결과가 xml 형식으로 반환된다. 이것을 dict 로 바꿔주는 라이브러리다
import os

filePath, fileName = os.path.split(__file__)
data_path = os.path.join(filePath,'using_data','기상청41_단기예보 조회서비스_오픈API활용가이드_격자_위경도(20221027).xlsx')


warnings.filterwarnings('ignore')
#초단기예보조회
#초단기예보정보를 조회하기 위해 발표일자, 발표시각, 예보지점 X 좌표, 예보지점 Y 좌표의 조회 조건으로 자료구분코드, 예보값, 발표일자, 발표시각, 예보지점 X 좌표, 예보지점 Y 좌표의 정보를 조회하는 기능
serviceKey='z5tZeY7iv8p2Ib1ApuO7q//wKERNpofeMjmWiun9zefOVRmdNnZkj9DCIiZoMTf3fhj6CEToyLj94bjSGZ4q4A==' #api 키
pageNo = '1' #페이지번호
numOfRows = '100' #한 페이지 결과 수
dataType = 'json' #요청자료형식(XML/JSON) Default: XML
base_date = '20221216' #‘22년 8월 13일 발표
base_time='1300' #06시30분 발표(30분 단위)
df=pd.read_excel(data_path)
#df['T1H','RN1','SKY','UUU','VVV','REH','PTY','LGT','VEC','WSD']=0
print(df.info())
for idx in df.index:
    nx=df.loc[idx,'격자 X']
    ny=df.loc[idx,'격자 Y']
    url = 'https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst?serviceKey={}&pageNo={}&numOfRows={}&dataType={}&base_date={}&base_time={}&nx={}&ny={}'.format(serviceKey,pageNo,numOfRows,dataType,base_date,base_time,nx,ny)
    response = requests.get(url, verify=False)
    r_data = json.loads(response.text)
    try : 
        data = pd.DataFrame(r_data['response']['body']['items']['item'])  
    except :
        data = pd.DataFrame(r_data['response']['body']['items']['item'], index = [0])
    data.drop_duplicates(['baseDate','baseTime','category'],keep='first',inplace=True,ignore_index = True)
    for j in data.index:
        df.loc[idx,data.loc[j,'category']]=data.loc[j,'fcstValue']
    print(idx)
print(df)

NameError: name '__file__' is not defined

# 홍수정보

In [13]:
import requests
import pprint
import json
import pandas as pd

def flowsiteAPI_data(serviceKey, HydroType, DataType, DocumentType):
        url = f'https://api.hrfco.go.kr/{serviceKey}/{HydroType}/{DataType}{DocumentType}'
        response = requests.get(url)
        contents = response.text
        json_ob = json.loads(contents)
        body = json_ob['content']
        body = pd.json_normalize(body)
        return body

def flowsiteAPI_livedata(serviceKey, HydroType, DataType, DocumentType):
        url = f'https://api.hrfco.go.kr/{serviceKey}/{HydroType}/{DataType}/10M{DocumentType}'
        response = requests.get(url)
        contents = response.text
        pp = pprint.PrettyPrinter(indent =4)
        # pp.pprint(response.content)
        json_ob = json.loads(contents)
        body = json_ob['content']
        # body = json_ob['response']['body']['item']
        body = pd.json_normalize(body)
        return body

def request_rainfall_info():
        serviceKey = 'A3A7BEB0-361E-4134-878C-BD8004204558'
        HydroType = 'rainfall'
        DataType = 'info'
        DocumentType = '.json'
        df = flowsiteAPI_data(serviceKey, HydroType, DataType, DocumentType)
        df['시도명'] = df['addr'].str.split(' ').str[0]
        df['시군구명'] = df['addr'].str.split(' ').str[1]
        df = df[df['lat'] != ' ']
        df['lat'] = df['lat'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
        df['lon'] = df['lon'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
        df.reset_index(drop = True, inplace = True)
        df.to_csv('강수량제원정보.csv', index = False, encoding = 'utf-8-sig')
        return df

def request_rainfall_live():
        serviceKey = 'A3A7BEB0-361E-4134-878C-BD8004204558'
        HydroType = 'rainfall'
        DataType = 'list'
        DocumentType = '.json'
        live_df = flowsiteAPI_livedata(serviceKey, HydroType, DataType, DocumentType)
        df = pd.read_csv('C:/pythonproject/streamlit (2)/pages/using_data/강수량제원정보.csv', encoding = 'utf-8-sig')
        live_df['rfobscd'] = live_df['rfobscd'].astype('int64')
        rainfall_df = pd.merge(df, live_df, on = 'rfobscd', how = 'left')
        rainfall_df[['lon','lat']] = rainfall_df[['lon','lat']].astype('float64')
        return rainfall_df

In [14]:
serviceKey = 'A3A7BEB0-361E-4134-878C-BD8004204558'
HydroType = 'waterlevel'
DataType = 'list'
DocumentType = '.json'
water_level_live = flowsiteAPI_livedata(serviceKey, HydroType, DataType, DocumentType)
DataType = 'info'
water_level = flowsiteAPI_data(serviceKey, HydroType, DataType, DocumentType)
water_level = water_level[water_level['attwl'] != ' ']
water_level['시도명'] = water_level['addr'].str.split(' ').str[0]
water_level['lat'] = water_level['lat'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
water_level['lon'] = water_level['lon'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
# data = pd.merge(water_level, water_level_live, on = 'wlobscd', how = 'inner')
# data = data[data['almwl'] != ' ']
# data[['attwl', 'wrnwl', 'almwl', 'srswl', 'wl']] = data[['attwl', 'wrnwl', 'almwl', 'srswl', 'wl']].astype('float64')
# data['누구'] = data.apply(lambda x : '심각 수위 단계' if x['wl'] >= x['srswl']
#                                                     else ('경보수위 단계' if x['wl'] >= x['almwl']
#                                                     else ('주의보수위 단계' if x['wl'] >= x['wrnwl']
#                                                     else ('관심수위 단계' if x['wl'] >= x['attwl'] else '정상수위 단계')))  , axis = 1)

In [15]:
water_level.reset_index(drop = True).to_csv('C:/pythonproject/streamlit (2)/pages/using_data/flowsiteAPI_data.csv', index = False, encoding = 'utf-8-sig')

OSError: Cannot save file into a non-existent directory: 'C:\pythonproject\streamlit (2)\pages\using_data'

In [ ]:
water_level = flowsiteAPI_data(serviceKey, HydroType, DataType, DocumentType)
water_level = water_level[water_level['attwl'] != ' ']
water_level['시도명'] = water_level['addr'].str.split(' ').str[0]
water_level['lat'] = water_level['lat'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
water_level['lon'] = water_level['lon'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
water_level.to_csv('flowsiteAPI_data.csv', index = False, encoding = 'utf-8-sig')

In [16]:
water_level = pd.read_csv('C:/pythonproject/streamlit (2)/pages/using_data/flowsiteAPI_data.csv')
water_level['wlobscd'] = water_level['wlobscd'].astype('object')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/pythonproject/streamlit (2)/pages/using_data/flowsiteAPI_data.csv'

In [17]:
water_level.to_csv('수위단계.csv', index = False, encoding = 'utf-8-sig')

In [18]:
serviceKey = 'A3A7BEB0-361E-4134-878C-BD8004204558'
HydroType = 'rainfall'
DataType = 'list'
DocumentType = '.json'
water_level_live = flowsiteAPI_livedata(serviceKey, HydroType, DataType, DocumentType)
DataType = 'info'
water_level = flowsiteAPI_data(serviceKey, HydroType, DataType, DocumentType)
# water_level = water_level[water_level['attwl'] != ' ']
water_level['시도명'] = water_level['addr'].str.split(' ').str[0]
water_level['lat'] = water_level['lat'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
water_level['lon'] = water_level['lon'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
data = pd.merge(water_level, water_level_live, on = 'rfobscd', how = 'inner')
data = data[data['agcnm'] == '환경부'].reset_index(drop = True)
# data = data[data['almwl'] != ' ']

In [19]:
data['시도명'].unique()

array(['강원도', '충청북도', '경상북도', '경기도', '서울특별시', '충청남도', '대구광역시', '경상남도',
       '전라북도', '부산광역시', '울산광역시', '대전광역시', '세종특별자치시', '전라남도', '광주광역시',
       '전남'], dtype=object)

In [ ]:
print('hi')

In [24]:
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FloatImage

m = folium.Map(
    location=[37, 127],
    zoom_start=8
)

coords = data[['lat', 'lon']]


# marker_cluster = MarkerCluster().add_to(m)

for lat, long in zip(coords['lat'], coords['lon']):
    folium.Marker([lat, long], icon = folium.Icon(color="green")).add_to(m)

image = ('C:/Users/82106/Documents/PythonProject/Disaster2_web/pages/using_data/수위.png')
image = Image.open(os.path.join('C:/Users/82106/Documents/PythonProject/Disaster2_web/pages/using_data/수위.png'))
FloatImage(image, bottom=18, left=1).add_to(m)

print(m)

In [ ]:
from streamlit_folium import st_folium

In [ ]:
serviceKey = 'A3A7BEB0-361E-4134-878C-BD8004204558'
HydroType = 'waterlevel'
DataType = 'list'
DocumentType = '.json'
water_level_live = flowsiteAPI_livedata(serviceKey, HydroType, DataType, DocumentType)
DataType = 'info'
water_level = pd.read_csv('C:/pythonproject/streamlit (2)/pages/using_data/flowsiteAPI_data.csv')
water_level_live['wlobscd'] = water_level_live['wlobscd'].astype('int64')
data = pd.merge(water_level, water_level_live, on = 'wlobscd', how = 'inner')
data = data[data['almwl'] != ' ']
data[['attwl', 'wrnwl', 'almwl', 'srswl', 'wl']] = data[['attwl', 'wrnwl', 'almwl', 'srswl', 'wl']].astype('float64')
data['수위경보'] = data.apply(lambda x : '심각 수위 단계' if x['wl'] >= x['srswl']
                                                    else ('경보수위 단계' if x['wl'] >= x['almwl']
                                                    else ('주의보수위 단계' if x['wl'] >= x['wrnwl']
                                                    else ('관심수위 단계' if x['wl'] >= x['attwl'] else '정상수위 단계')))  , axis = 1)
data 

In [ ]:
serviceKey = 'A3A7BEB0-361E-4134-878C-BD8004204558'
HydroType = 'waterlevel'
DataType = 'list'
DocumentType = '.json'
water_level_live = flowsiteAPI_livedata(serviceKey, HydroType, DataType, DocumentType)
DataType = 'info'

data = pd.merge(water_level, water_level_live, on = 'wlobscd', how = 'inner')
data = data[data['almwl'] != ' ']
data[['attwl', 'wrnwl', 'almwl', 'srswl', 'wl']] = data[['attwl', 'wrnwl', 'almwl', 'srswl', 'wl']].astype('float64')
data['수위경보'] = data.apply(lambda x : '심각 수위 단계' if x['wl'] >= x['srswl']
                                                    else ('경보수위 단계' if x['wl'] >= x['almwl']
                                                    else ('주의보수위 단계' if x['wl'] >= x['wrnwl']
                                                    else ('관심수위 단계' if x['wl'] >= x['attwl'] else '정상수위 단계')))  , axis = 1)

In [ ]:
water_level = flowsiteAPI_data(serviceKey, HydroType, DataType, DocumentType)
# water_level = water_level[water_level['attwl'] != ' ']
# water_level['시도명'] = water_level['addr'].str.split(' ').str[0]
water_level['lat'] = water_level['lat'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
water_level['lon'] = water_level['lon'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)

In [ ]:
water_level = water_level[(water_level['lon'] != ' ') & (water_level['lat'] != ' ')]
water_level['lat'] = water_level['lat'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
water_level['lon'] = water_level['lon'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)

In [ ]:
water_level = water_level[(water_level['lon'] != ' ') & (water_level['lat'] != ' ')]

In [ ]:
water_level

In [ ]:
def rainfall_api():
    serviceKey = 'A3A7BEB0-361E-4134-878C-BD8004204558'
    HydroType = 'rainfall'
    DataType = 'list'
    DocumentType = '.json'
    water_level_live = flowsiteAPI_livedata(serviceKey, HydroType, DataType, DocumentType)
    DataType = 'info'
    water_level = flowsiteAPI_data(serviceKey, HydroType, DataType, DocumentType)
    water_level['시도명'] = water_level['addr'].str.split(' ').str[0]
    water_level['lat'] = water_level['lat'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
    water_level['lon'] = water_level['lon'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
    rainfall_df = pd.merge(water_level, water_level_live, on = 'rfobscd', how = 'inner')
    return rainfall_df

df = rainfall_api()

In [ ]:
import folium
def rainfall_map(data):
    m = folium.Map(
    location=[data['lat'].mean(), data['lon'].mean()],
    zoom_start= 7
    )
    coords = data[['lat', 'lon', 'obsnm', 'rf']]
    for idx in coords.index:
        folium.Marker([coords.loc[idx, 'lat'], coords.loc[idx, 'lon']], icon = folium.Icon(color="green"), tooltip = coords.loc[idx,'obsnm'] + ' : ' + str(coords.loc[idx,'rf'])).add_to(m)
    return m
rainfall_map(df)

In [ ]:
from datetime import datetime
def flood_news(serviceKey, HydroType, DataType,Edt, DocumentType):
        url = f'https://api.hrfco.go.kr/{serviceKey}/{HydroType}/{DataType}/{Edt}{DocumentType}'
        response = requests.get(url)
        contents = response.text
        json_ob = json.loads(contents)
        body = json_ob['content']
        body = pd.json_normalize(body)
        return body
    
serviceKey = 'A3A7BEB0-361E-4134-878C-BD8004204558'
HydroType = 'fldfct'
DataType = 'list'
Edt = '20220906'
DocumnetType = '.json'
print(datetime.today().strftime(("%Y%m%d")))
df = flood_news(serviceKey, HydroType, DataType, Edt, DocumentType).drop(columns = 'links')
df.columns = ['발표일시','발표자','수위도달 예상일시', '예상 수위표수위', '예상 해발수위', '홍수예보 종류', '홍수예보 번호', '지점', '기존발령일시', 
            '비고','강명','변동상황', '현재 일시', '현재 수위표수위', '현재 해발수위', '예상 일시(변동)', '예상 수위표수위(변동)', '예상 해발수위(변동)', '관측소 코드', '주의 지역', 
            '주의 강명']

In [ ]:
list_ = []
for idx in df.index:
    if df.loc[idx,'홍수예보 종류'][-2:] == '발령':
        list_.append(df.loc[idx, '주의 지역'])
list_

In [ ]:
import requests
import pprint
import json
import pandas as pd

def damAPI_data(serviceKey, HydroType, DataType, DocumentType):
        url = f'https://api.hrfco.go.kr/{serviceKey}/{HydroType}/{DataType}{DocumentType}'
        response = requests.get(url)
        contents = response.text
        json_ob = json.loads(contents)
        body = json_ob['content']
        body = pd.json_normalize(body)
        return body

def dam_livedata(serviceKey, HydroType, DataType, DocumentType):
        url = f'https://api.hrfco.go.kr/{serviceKey}/{HydroType}/{DataType}/1H{DocumentType}'
        response = requests.get(url)
        contents = response.text
        pp = pprint.PrettyPrinter(indent =4)
        # pp.pprint(response.content)
        json_ob = json.loads(contents)
        body = json_ob['content']
        # body = json_ob['response']['body']['item']
        body = pd.json_normalize(body)
        return body

def dam_data_make():
        serviceKey = 'A3A7BEB0-361E-4134-878C-BD8004204558'
        HydroType = 'dam'
        DataType = 'list'
        DocumnetType = '.json'
        livedata = flowsiteAPI_livedata(serviceKey, HydroType, DataType, DocumentType)
        DataType = 'info'
        data = flowsiteAPI_data(serviceKey, HydroType, DataType, DocumentType)
        dam_data = pd.merge(data, livedata, on = 'dmobscd', how = 'inner')
        dam_data = dam_data[dam_data['lat'] != ' ']
        dam_data = dam_data[dam_data['lon'] != ' ']
        dam_data['lat'] = dam_data['lat'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
        dam_data['lon'] = dam_data['lon'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
        return dam_data

def dam_map(data):
        m = folium.Map(
        location=[data['lat'].mean(), data['lon'].mean()],
        zoom_start= 7
        )
        coords = data[['lat', 'lon', 'obsnm', 'swl', 'inf', 'sfw', 'ecpc', 'tototf']]
        for idx in coords.index:
                text = coords.loc[idx,'obsnm']+ '<br>현재 수위 :' + str(coords.loc[idx,'swl']) + '<br>유입량 :' + str(coords.loc[idx,'inf'])+ '<br>저수량 :' + str(coords.loc[idx,'sfw']) + '<br>공용량 :' + str(coords.loc[idx,'ecpc']) + '<br>총 방류량 :' + str(coords.loc[idx,'tototf'])
                folium.Marker([coords.loc[idx, 'lat'], coords.loc[idx, 'lon']], icon = folium.Icon(color="purple"), tooltip = text).add_to(m)
        return m

In [ ]:
import requests
import pprint
import json
import pandas as pd

def bo_data_make():
        serviceKey = 'A3A7BEB0-361E-4134-878C-BD8004204558'
        HydroType = 'bo'
        DataType = 'list'
        DocumnetType = '.json'
        livedata = flowsiteAPI_livedata(serviceKey, HydroType, DataType, DocumentType)
        DataType = 'info'
        data = flowsiteAPI_data(serviceKey, HydroType, DataType, DocumentType)
        bo_data = pd.merge(data, livedata, on = 'boobscd', how = 'inner')
        bo_data = bo_data[bo_data['lat'] != ' ']
        bo_data = bo_data[bo_data['lon'] != ' ']
        bo_data['lat'] = bo_data['lat'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
        bo_data['lon'] = bo_data['lon'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
        return bo_data

def bo_map(data):
        m = folium.Map(
        location=[data['lat'].mean(), data['lon'].mean()],
        zoom_start= 7
        )
        coords = data[['lat', 'lon', 'obsnm', 'swl', 'inf', 'sfw', 'ecpc', 'tototf']]
        for idx in coords.index:
                text = coords.loc[idx,'obsnm']+ '<br>현재 수위 :' + str(coords.loc[idx,'swl']) + '<br>유입량 :' + str(coords.loc[idx,'inf'])+ '<br>저수량 :' + str(coords.loc[idx,'sfw']) + '<br>공용량 :' + str(coords.loc[idx,'ecpc']) + '<br>총 방류량 :' + str(coords.loc[idx,'tototf'])
                folium.Marker([coords.loc[idx, 'lat'], coords.loc[idx, 'lon']], icon = folium.Icon(color="purple"), tooltip = text).add_to(m)
        return m

In [ ]:
data = bo_data_make()
bo_map(data)